In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import re
import json

import numpy as np 
import pandas as pd 

import glob

import matplotlib.pyplot as plt

In [2]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
train_df.sample(2)

,Id,pub_title,dataset_title,dataset_label,cleaned_label
703,b76fb7f9-338f-41a9-9db3-4a6d37313ee6,Prostate-Specific Antigen Testing of Older Men,Baltimore Longitudinal Study of Aging (BLSA),Baltimore Longitudinal Study of Aging (BLSA),baltimore longitudinal study of aging blsa
11113,c6ee1b1d-76ba-4f29-a998-34b4cc84231d,Small Differences that Matter: Mistakes in App...,Beginning Postsecondary Student,Beginning Postsecondary Students,beginning postsecondary students


In [3]:
sub_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
sub_df.sample(3)

,Id,PredictionString
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN
0,2100032a-7c33-4bff-97ef-690822c43466,NaN


In [4]:
train_df.columns

Index(['Id', 'pub_title', 'dataset_title', 'dataset_label', 'cleaned_label'], dtype='object')

In [5]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [6]:
title1 = [temp_title.lower() for temp_title in train_df['dataset_label'].unique()]
label1 = [temp_label.lower() for temp_label in train_df['dataset_title'].unique()]
clabel1 = [temp_cleanLabel.lower() for temp_cleanLabel in train_df['cleaned_label'].unique()]

all_labels1 = list(set(list(map(clean_text, set(title1))) + list(map(clean_text, set(label1)))))

In [7]:
train_files = glob.glob("../input/coleridgeinitiative-show-us-the-data/test/*.json")
train_files

['../input/coleridgeinitiative-show-us-the-data/test/8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60.json',
 '../input/coleridgeinitiative-show-us-the-data/test/2100032a-7c33-4bff-97ef-690822c43466.json',
 '../input/coleridgeinitiative-show-us-the-data/test/2f392438-e215-4169-bebf-21ac4ff253e1.json',
 '../input/coleridgeinitiative-show-us-the-data/test/3f316b38-1a24-45a9-8d8c-4e05a42257c6.json']

In [8]:
test_files = glob.glob("../input/coleridgeinitiative-show-us-the-data/test/*.json")

In [9]:
test_df = pd.DataFrame()

for test_file in test_files:
    text_data=pd.read_json(test_file)
    text_data.insert(0,'id', test_file.split('/')[-1].split('.')[0]) 
    test_df = pd.concat([test_df, text_data])

In [10]:
test_df.head()

,id,section_title,text
0,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Introduction,A significant body of research has been conduc...
1,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Literature review,We reviewed the literature that explored retai...
2,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Food shopping patterns: where do people shop?,Diversification in the food retail sector offe...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Food shopping patterns: what do people buy?,"Many factors, including income, participation ..."
4,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,2,Anne Palmer et al. shopping at the same store ...


In [11]:
test_df['cleaned_text'] = test_df.apply(lambda txt: clean_text(txt['text']).strip(), axis =1)

In [12]:
test_df.head()

,id,section_title,text,cleaned_text
0,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Introduction,A significant body of research has been conduc...,a significant body of research has been conduc...
1,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Literature review,We reviewed the literature that explored retai...,we reviewed the literature that explored retai...
2,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Food shopping patterns: where do people shop?,Diversification in the food retail sector offe...,diversification in the food retail sector offe...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,Food shopping patterns: what do people buy?,"Many factors, including income, participation ...",many factors including income participation in...
4,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,2,Anne Palmer et al. shopping at the same store ...,anne palmer et al shopping at the same store h...


In [13]:
test_group = pd.DataFrame(test_df.groupby(['id'])['cleaned_text'].agg(' '.join))

test_group.head()

,cleaned_text
id,
2100032a-7c33-4bff-97ef-690822c43466,cognitive deficits and reduced educational ach...
2f392438-e215-4169-bebf-21ac4ff253e1,this report describes how the education system...
3f316b38-1a24-45a9-8d8c-4e05a42257c6,cape hatteras national seashore caha located a...
8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,a significant body of research has been conduc...


In [14]:
result = test_group['cleaned_text'].apply(lambda txt: '|'.join({label for label in all_labels1 if label in txt})).reset_index()

In [15]:
result.columns =['Id', 'PredictionString']
result

,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,alzheimer s disease neuroimaging initiative ad...
1,2f392438-e215-4169-bebf-21ac4ff253e1,nces common core of data|common core of data|t...
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,slosh model|sea lake and overland surges from ...
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,rural urban continuum codes


In [16]:
result.to_csv('submission.csv', index=False)